<a href="https://colab.research.google.com/github/Vaycold/pytorch_tutorial/blob/main/%2300.%EB%B9%A0%EB%A5%B8%EC%8B%9C%EC%9E%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 빠른 시작


### 데이터 작업하기

    데이터 작업을 위한 기본 요소 두가지
    1) torch.utils.data.DataLoader : Dataset을 반복가능한 객체로 감쌈
    2) torch.utils.data.Dataset    : 샘플과 정답을 저장



In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [2]:
# 모든 Torchvision dataset은 샘플과 정답을 각각 변경하기 위한 transform과 target_transfor 두 인자를 포함함

training_data  = datasets.FashionMNIST(
    root = 'data',
    train= True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = 'data',
    train= False,
    download = True,
    transform = ToTensor()
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
# Dataset을 Dataloader의 인자로 전달
# -> 반복가능한 객체로 감싸고 자동화된 배치, 샘플링, 섞기 등 다중 프로세스로 데이터 불러오기 지원
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

for x,y in test_dataloader :
    print('shape of x [N, C, H, W] : ', x.shape)
    print('shape of y : ', y.shape, y.dtype)
    break

shape of x [N, C, H, W] :  torch.Size([64, 1, 28, 28])
shape of y :  torch.Size([64]) torch.int64


### 모델만들기

    : PyTorch에서 신경망 모델은 nn.Module을 상속받는 클래스를 생성하여 정의.
    : __init__함수에서 신경망의 계층들을 정의한 후 forward 함수에서 신경망에 데이터를 어떻게 전달할지 지정
    : 가능한 경우 GPU로 신경망을 이동시켜 연산 가속화


In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('using {} device'.format(device))

class NeuralNetwork(nn.Module) :
    def __init__(self) :
        super(NeuralNetwork,self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
            nn.ReLU()
        )
    def forward(self, x) :
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


### 모델 매개변수 최적화

    : 손실함수와 옵티마이저 필요

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=1e-3)

In [9]:
def train(dataloader, model, loss_fn, optimizer) :
    size = len(dataloader.dataset)
    for batch, (x,y) in enumerate(dataloader) :
        x, y = x.to(device), y.to(device)

        pred = model(x)
        loss = loss_fn(pred,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0 :
            loss, current = loss.item(), batch*len(x)
            print(f'loss : {loss:>7f} [{current :>5d}/{size:>5d}]')

In [10]:
def test(dataloader, model, loss_fn) :
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad() :
        for x,y in dataloader :
            x,y = x.to(device), y.to(device)
            pred = model(x)
            test_loss += loss_fn(pred,y ).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
epochs = 5
for i in range(epochs) :
    print(f"Epoch {i+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss : 2.299272 [    0/60000]
loss : 2.288500 [ 6400/60000]
loss : 2.279526 [12800/60000]
loss : 2.285094 [19200/60000]
loss : 2.259532 [25600/60000]
loss : 2.243056 [32000/60000]
loss : 2.251852 [38400/60000]
loss : 2.219608 [44800/60000]
loss : 2.222190 [51200/60000]
loss : 2.221250 [57600/60000]
Test Error: 
 Accuracy: 49.8%, Avg loss: 2.197175 

Epoch 2
-------------------------------
loss : 2.197404 [    0/60000]
loss : 2.192135 [ 6400/60000]
loss : 2.147911 [12800/60000]
loss : 2.183201 [19200/60000]
loss : 2.122777 [25600/60000]
loss : 2.091721 [32000/60000]
loss : 2.135859 [38400/60000]
loss : 2.055960 [44800/60000]
loss : 2.069557 [51200/60000]
loss : 2.069826 [57600/60000]
Test Error: 
 Accuracy: 54.1%, Avg loss: 2.015799 

Epoch 3
-------------------------------
loss : 2.024051 [    0/60000]
loss : 2.010616 [ 6400/60000]
loss : 1.905664 [12800/60000]
loss : 1.993167 [19200/60000]
loss : 1.868321 [25600/60000]
loss : 1.824304 [32000/600

### 모델 저장하기

    : 모델을 저장하는 일반적인 방법은 모델의 매개변수들을 포함하여 내부 상태사전을 직렬화하는 것

In [12]:
torch.save(model.state_dict(), 'model.pth')

### 모델 불러오기

    : 모델을 불러오는 과정에는 모델 구조를 다시 만들고 상태사전을 모델에 불러오는 과정이 포함됨
    

In [13]:
model = NeuralNetwork()
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>